# PostgreSQL Database

In [1]:
import psycopg2
import os

In [2]:
DB_PASSWORD = os.environ['DB_PASSWORD']

In [3]:
conn_local = psycopg2.connect(database="cookit", user = "lillykaemmerling", host = "localhost", port = "5432")

In [4]:
cur_local  = conn_local.cursor()

In [ ]:
#Weirdly for me the user is my name normally it should be postgres by default 
conn = psycopg2.connect(database="d1hsr1c7nk56dl", user = "iadkkqrgljveni", host = "ec2-3-230-61-252.compute-1.amazonaws.com", port = "5432", password=DB_PASSWORD)

In [ ]:
cur = conn.cursor()

In [ ]:
#Creating the table only ONCE
cur_local.execute(
    '''CREATE TABLE RECIPES_NEW
    (ID INT PRIMARY KEY     NOT NULL,
    TITLE           TEXT    NOT NULL,
    DIFFICULTY      TEXT,
    PREPTIME        TEXT,
    LINK TEXT  NOT NULL,
    PICTURE_URL     TEXT,
    INGREDIENTS     TEXT[]         NOT NULL,
    CUISINE         TEXT[]     NOT NULL);''')

In [ ]:
#In Postgres we always have to commit 
conn_local.commit()

# Scraping lewagon recipes 

In [5]:
from bs4 import BeautifulSoup
import requests
import csv

## Scrape recipe overview

In [6]:
#Scrape the main pages content 
def scrape_from_internet(pages):

    #USE f string to do this for all 63 pages 
    BASIC_URI = "https://recipes.lewagon.com/"
    html_content = bytearray()
    
    #Pagination --> start refers to the page
    for i in range(1,pages):
        url = f"{BASIC_URI}?search[query]=&page={i}"
        
        response = requests.get(url)

        if response.history == []:
            html_content += response.content

    return str(html_content)

In [7]:
#parse the main page content 
def parse(html):
    #Return a list from the overview page 
    #html input = response.content

    soup = BeautifulSoup(html, 'html.parser')
    #Find all recipes on the specific page
    recipes_soup = soup.find_all('div', class_='col-12 col-sm-6 col-md-4 col-lg-3')
    
    recipe_list = []

    for recipe in recipes_soup:
        name = recipe.find('p', class_='text-dark text-truncate w-100 font-weight-bold mb-0 recipe-name').text
        difficulty = recipe.find('span', class_='recipe-difficulty').text
        prep_time = recipe.find('span', class_='recipe-cooktime').text
        
        #Get the link from data-href to be able to go to the recipe page 
        recipe_url = recipe.find('div', class_='recipe my-3').attrs['data-href']
        
        #Get the link to the picture from src 
        picture_url = recipe.find('img', class_='recipe-img').attrs['src']

        recipe_list.append({"name": name, 
                            "difficulty": difficulty, 
                            "prep_time": prep_time,
                            "link": recipe_url,
                            "picture_url": picture_url})
    return recipe_list

In [ ]:
recipes = parse(scrape_from_internet(64))

In [ ]:
#write the parsed content into a csv 
def write_csv(recipes):
    #dump recipes to a CSV file `recipes.csv`
    with open(f"../raw_data/recipes.csv", "w") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=recipes[0].keys())
        writer.writeheader()
        for recipe in recipes:
            writer.writerow(recipe)

In [ ]:
write_csv(recipes)

Now we have the overview of all 750 recipes in a csv
As a next step we need to go through this csv/the original dictionnary and scrape every individual recipe link in order to access ingredients & cuisine 

## Scrape individual recipes 

In [8]:
import pandas as pd

In [9]:
#scrape the individual recipe 
def scrape_recipe(url):

    html_content = bytearray()
    response = requests.get(url)

    if response.history == []:
        html_content += response.content

    return str(html_content)

In [10]:
def parse(recipe_html):
    
    #html input = response.content

    soup = BeautifulSoup(recipe_html, 'html.parser')
    details = {}
    
    #Find all ingredients on the specific page
    ingredients_soup = soup.find_all('div', class_='ingredient')
    ingredients = []

    for ingredient in ingredients_soup:
        description = ingredient.find('p', class_='mb-0').text
        ingredients.append(description)
        
    
    #Find all ingredients on the specific page
    cuisines = []
    cuisines_soup = soup.find_all('span', class_="badge badge-success")
    
    if cuisines_soup == None:
        cuisines.append("No cuisine specified")
    else:
        for cuisine in cuisines_soup:
            cuisines.append(cuisine.text)
    
    details["ingredients"] = ingredients
    details["cuisine"] = cuisines
    
    return details

In [11]:
recipe = parse(scrape_recipe("https://recipes.lewagon.com/recipes/548"))

In [12]:
recipe["ingredients"]

['2 c. (12 oz. pkg.) semi-sweet chocolate chips',
 '14 oz. can sweetened condensed milk',
 '1 t. vanilla extract',
 '1/8 t. salt',
 '3 c. mini-marshmallows',
 '1 1/2 c. toasted, chopped walnuts']

TODO: 
+ Insert insert into csv/ db where this url is present 
+ additionally split amount and ingredient 

In [13]:
recipes = pd.read_csv("../raw_data/recipes.csv")

In [14]:
recipes.head()

,name,difficulty,prep_time,link,picture_url
0,10 Minute Brownies,Moderate,45 min,https://recipes.lewagon.com/recipes/668,https://spoonacular.com/recipeImages/631830-55...
1,15 Minute Healthy Dark Chocolate Truffles,Very hard,45 min,https://recipes.lewagon.com/recipes/956,https://spoonacular.com/recipeImages/631841-55...
2,4 Ingredient Chicken Pot Pie,Very easy,45 min,https://recipes.lewagon.com/recipes/970,https://spoonacular.com/recipeImages/631868-55...
3,4 Ingredient Raw Peanut Butter Chocolate Cups,Moderate,45 min,https://recipes.lewagon.com/recipes/748,https://spoonacular.com/recipeImages/792705-55...
4,5-Minute Blueberry Chia Jam,Moderate,45 min,https://recipes.lewagon.com/recipes/523,https://spoonacular.com/recipeImages/631880-55...


## Insert Recipes into database 

In [ ]:
#Based on overview csv file 
def insert_into_db(recipes):
    
    for index, row in recipes.iterrows():
        primary_key = index
        title = row["name"]
        difficulty = row["difficulty"]
        prep_time = row["prep_time"]
        url = row["link"]
        picture_url = row["picture_url"]

        recipe = parse(scrape_recipe(url))

        ingredients = recipe["ingredients"]
        cuisine = recipe["cuisine"]
        
        query = """INSERT INTO RECIPES_NEW (ID, TITLE, DIFFICULTY, PREPTIME, LINK, PICTURE_URL, INGREDIENTS, CUISINE)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"""
        
        cur_local.execute(query, (primary_key, title, difficulty, prep_time, url, picture_url, ingredients, cuisine));
        conn_local.commit()

In [ ]:
insert_into_db(recipes.head(20))

## Testing querying local db 

In [19]:
cur_local.execute('''SELECT *
FROM recipes_new;''')

In [20]:
rows = cur_local.fetchall()

In [21]:
for row in rows:
    print(row)

(0, '10 Minute Brownies', 'Moderate', '45 min', 'https://recipes.lewagon.com/recipes/668', 'https://spoonacular.com/recipeImages/631830-556x370.jpg', ['1 teaspoon baking powder', '1/4 cup canola oil', '3/4 cup finely-chopped walnuts', '1 cup semisweet chocolate chips', '3/4 cup sugar', '1 cup unbleached flour', '1/2 cup unsweetened applesauce', '1 teaspoon pure vanilla extract'], ['American'])
(1, '15 Minute Healthy Dark Chocolate Truffles', 'Very hard', '45 min', 'https://recipes.lewagon.com/recipes/956', 'https://spoonacular.com/recipeImages/631841-556x370.jpg', ['15 dried dates, pitted', '3/4 cups sliced or slivered almonds', '1/2 cup pecans', '1/4 cup plus 2 tablespoons cocoa powder', '1 teaspoon vanilla extract', '1 tablespoon agave nectar (or honey)', '1/4 teaspoon sea salt', '1 cup hot water', 'Optional: coconut flakes, sprinkles, cocoa powder, or crushed nuts for coating the truffles'], [])
(2, '4 Ingredient Chicken Pot Pie', 'Very easy', '45 min', 'https://recipes.lewagon.com/